# Start Training

## Model and Dataset Class

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import copy    #For saving the MOST ACCURATE MODEL 
from PIL import Image
import pandas as pd
import os
import torch
from dotenv import load_dotenv
import torch.nn as nn
load_dotenv()

train_labels_csv_dir = os.path.join(os.getenv("dataset_dir"),"boards","train","labels.csv")
val_labels_csv_dir = os.path.join(os.getenv("dataset_dir"),"boards","val","labels.csv")

train_dataset_dir = os.path.join(os.getenv("dataset_dir"),"boards","train","images")
val_dataset_dir = os.path.join(os.getenv("dataset_dir"),"boards","val","images")


class BoardKeypointDataset(Dataset):
    def __init__(self,csv_file, root_dir, transform=None, normalize=True):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.normalize = normalize

    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])

        image = Image.open(img_name).convert("RGB")

        keypoints = self.annotations.iloc[idx, 1:].values.astype('float32')

        w, h = image.size

        if self.normalize:
            keypoints[::2] /= w  # x'ler
            keypoints[1::2] /= h  # y'ler

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(keypoints)

class BoardKeypointModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(          #The input images are sized as 480 x 480.  
            nn.Conv2d(3,16,5,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 60 * 60), #After 3 2x2 max pooling, the image sizes are now 60 x 60
            nn.ReLU(),
            nn.Linear(128, 8),  # 4 keypoint = 8 sayı
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

train_transform = transforms.Compose([
    transforms.Resize((480,480)),
    transforms.GaussianBlur((3,3),[0.1,0.5]),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

## Training

In [ ]:
import time

train_dataset = BoardKeypointDataset(train_labels_csv_dir, train_dataset_dir, transform=train_transform)
val_dataset = BoardKeypointDataset(val_labels_csv_dir, val_dataset_dir, transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=16, num_workers=4 ,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=4 ,shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BoardKeypointModel().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

lowest = 1e15

start_time = time.time()

for epoch in range(150):
    
    model.train()
    running_loss = 0
    
    for i,(imgs,targets) in enumerate(train_loader):
        imgs, targets = imgs.to(device), targets.to(device)

        preds = model(imgs)
        loss = criterion(preds, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 10 == 9:
            avg_train_loss = running_loss / len(train_loader)
            print(f"[Epoch {epoch+1}] Avg Train Loss: {avg_train_loss:.4f} | Avg Val Loss: {avg_val_loss:.4f}")



    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs,targets in val_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            preds = model(imgs)
            loss = criterion(preds,targets)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    print(f"[Epoch {epoch+1}] Average Val Loss: {avg_val_loss:.4f}")

    torch.save( model.state_dict() ,os.getenv("models_dir") + "/last.pth")

    if lowest - avg_val_loss > 1e-5:        # Didn't want to save everything. I decided a threshold value of 1e-5. If there's an improvement bigger than that, the new model will be saved.
        lowest = avg_val_loss
        most_accurate_model = copy.deepcopy(model.state_dict())
        torch.save(most_accurate_model, os.getenv("models_dir") + "/best.pth")
        continue


print(f"The Best Loss: {lowest}%")
print(time.strftime('%H:%M:%S', time.gmtime( time.time() - start_time )))

            